In [30]:
import pandas as pd
import numpy as np
import sqlite3

data_dir = 'R6/'
engine = sqlite3.connect('yahoo')

In [92]:
# Unbiased Offline Evaluation of Contextual-bandit-based News Article Recommendation Algorithms 
# https://arxiv.org/pdf/1003.5956.pdf

f = open(data_dir + 'clicks_1.txt')

In [48]:
line = f.readline().strip()
parts = line.split('|')

In [49]:
unix_timestamp, disp_article_id, clicked = parts[0].split()
print('Unix Timestamp: {0}, Displayed Article Id: {1}, Clicked: {2}'.format(unix_timestamp, disp_article_id, clicked))

Unix Timestamp: 1241160900, Displayed Article Id: 109484, Clicked: 0


In [119]:
articles_df = pd.DataFrame(columns=['2', '3', '4', '5', '6', '1'])

for i in range(2, len(parts)):
    # Extract article vector information.
    article_info = parts[i].split()
    article_id = article_info[0]
    article_info = list(map(lambda x : x.split(':')[1], article_info[1:]))
    article_info = dict(zip(articles_df.columns, article_info))
    articles_df.loc[article_id] = pd.Series(article_info)
    
articles_df
# articles_df.drop_duplicates()

,2,3,4,5,6,1
109498,0.306008,0.000450,0.077048,0.230439,0.386055,1.000000
109509,0.306008,0.000450,0.077048,0.230439,0.386055,1.000000
109508,0.264355,0.000012,0.037393,0.420649,0.277591,1.000000
109473,0.295442,0.000014,0.135191,0.292304,0.277050,1.000000
109503,0.306008,0.000450,0.077048,0.230439,0.386055,1.000000
109502,0.277121,0.000131,0.038153,0.335835,0.348760,1.000000
109501,0.249086,0.001009,0.514682,0.067732,0.167491,1.000000
109492,0.331830,0.000022,0.019904,0.440390,0.207855,1.000000
109495,0.313277,0.000125,0.018413,0.410555,0.257630,1.000000
109494,0.306008,0.000450,0.077048,0.230439,0.386055,1.000000


In [114]:
pd.Series(article_info,)

1     1.000000
2     0.306008
3     0.000450
4     0.077048
5     0.230439
6     0.386055
id      109498
dtype: object